In [1]:
import glob
import pandas as pd
import numpy as np
import itertools
import malariagen_data

In [2]:
# Get CSV files list from a folder
path = 'build/h12-signal-detection_backup'
csv_files = glob.glob(path + "/*.csv")

In [3]:
#check how many files are there? = there are 104 files, but after removing, just 93
len(csv_files)

93

In [4]:
csv_files

['build/h12-signal-detection_backup/GH-WP_Shama-Ahanta-East_colu_2012_Q3_2RL.csv',
 'build/h12-signal-detection_backup/CI-LG_Agneby-Tiassa_colu_2012_2RL.csv',
 'build/h12-signal-detection_backup/UG-E_Tororo_gamb_2012_Q4_3RL.csv',
 'build/h12-signal-detection_backup/ML-2_Kati_gamb_2014_Q3_3RL.csv',
 'build/h12-signal-detection_backup/BF-09_Houet_gamb_2012_Q3_X.csv',
 'build/h12-signal-detection_backup/GW-BM_Setor-De-Safim_gcx1_2010_2RL.csv',
 'build/h12-signal-detection_backup/AO-LUA_Luanda_colu_2009_Q2_X.csv',
 'build/h12-signal-detection_backup/BF-09_Houet_colu_2014_Q3_3RL.csv',
 'build/h12-signal-detection_backup/GN-F_Kissidougou_gamb_2012_Q4_X.csv',
 'build/h12-signal-detection_backup/TZ-13_Tarime_arab_2012_Q3_3RL.csv',
 'build/h12-signal-detection_backup/BF-09_Houet_colu_2014_Q3_2RL.csv',
 'build/h12-signal-detection_backup/TZ-26_Moshi_arab_2012_Q3_3RL.csv',
 'build/h12-signal-detection_backup/ML-3_Yanfolila_gamb_2012_Q4_X.csv',
 'build/h12-signal-detection_backup/TZ-26_Moshi_arab_

In [6]:
#make a shortlist of Uganda gambiae csv files
#Ugambiae_csv_files = glob.glob(path + "/UG-*_gamb*.csv")
all_gambiae_csv_files = glob.glob(path + "/*gamb*.csv")

In [7]:
len(all_gambiae_csv_files)

40

In [8]:
# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file) for file in all_gambiae_csv_files)
#for data in df_list:
#    print(data)
# Concatenate all Uganda DataFrames
all_gambiae_signals_df   = pd.concat(df_list, axis=0, ignore_index=True)
len(all_gambiae_signals_df)

222

In [10]:
#check chromosome lengths in malariagen data
ag3 = malariagen_data.Ag3()

In [11]:
#these need to be converted into the usual 2L, 3R etc contigs. Depending on whether the L or R arm is first, may need to revise this!
new_contig = []
for index,row in all_gambiae_signals_df.iterrows():
    if row['contig'] == '2RL' and row['focus_pstart'] > ag3.genome_sequence('2R').shape[0]:
        new_contig.append('2L')
        print('region is on 2L')
    elif row['contig'] == '2RL' and row['focus_pstart'] <= ag3.genome_sequence('2R').shape[0]:
        print('region is on 2R')
        new_contig.append('2R')
    elif row['contig'] == '3RL' and row ['focus_pstart'] > ag3.genome_sequence('3R').shape[0]:
        new_contig.append('3L')
        print('region is on 3L')
    elif row['contig'] == '3RL' and row ['focus_pstart'] <= ag3.genome_sequence('3R').shape[0]:
        new_contig.append('3R')
        print('region is on 3R')
    elif row['contig'] == 'X':
        new_contig.append('X')
        print('region is on X chromosome')
    else:
        print('oops out of range')
        
    

region is on 3R
region is on 3R
region is on 3R
region is on 3R
region is on 3R
region is on 3R
region is on 3R
region is on X chromosome
region is on X chromosome
region is on X chromosome
region is on X chromosome
region is on X chromosome
region is on X chromosome
region is on X chromosome
region is on 3R
region is on 3R
region is on 3R
region is on 3L
region is on 2R
region is on 2L
region is on 2L
region is on 2L
region is on 2L
region is on 2L
region is on 2R
region is on 2R
region is on 2R
region is on 2R
region is on 2R
region is on 2L
region is on 2L
region is on 2L
region is on 2L
region is on 3R
region is on 3R
region is on 3R
region is on 3L
region is on 3L
region is on 2R
region is on 2R
region is on 2R
region is on 2R
region is on 2L
region is on 2L
region is on 2L
region is on 2L
region is on 2L
region is on 2R
region is on 2R
region is on 2R
region is on 2L
region is on 2L
region is on 2L
region is on 2L
region is on 2L
region is on 2L
region is on 2L
region is on 2L
re

In [12]:
#add the new contig to the dataframe
all_gambiae_signals_df['new_contig'] = new_contig


In [13]:
len(new_contig)

222

In [14]:
#now work out the focus positions based on new contig
#X: same as before
#2L: subtract 60132453
#2R: as before
#3L: subtract 52226568
#3LR: as before
new_focus_pstart = []
new_focus_pstop = []
for index,row in all_gambiae_signals_df.iterrows():
    if row['new_contig'] == 'X':
        new_focus_pstart.append(row['focus_pstart'])
        new_focus_pstop.append(row['focus_pstop'])
    elif row['new_contig'] == '2L':
        new_focus_pstart.append(row['focus_pstart'] - ag3.genome_sequence('2R').shape[0])
        new_focus_pstop.append(row['focus_pstop'] - ag3.genome_sequence('2R').shape[0])
    elif row['new_contig'] == '2R':
        new_focus_pstart .append(row['focus_pstart'])
        new_focus_pstop .append(row['focus_pstop'])
    elif row['new_contig'] == '3L':
        new_focus_pstart.append(row['focus_pstart'] - ag3.genome_sequence('3R').shape[0])
        new_focus_pstop.append(row['focus_pstop'] - ag3.genome_sequence('3R').shape[0])
    elif row['new_contig'] == '3R':
        new_focus_pstart .append(row['focus_pstart'])
        new_focus_pstop .append(row['focus_pstop'])
    else:
        print('oops out of range')

In [15]:
all_gambiae_signals_df['new_focus_pstart'] = new_focus_pstart
all_gambiae_signals_df['new_focus_pstop'] = new_focus_pstop
all_gambiae_signals_df.head()


,cohort_id,contig,gcenter,pcenter,delta_i,stat_max,gpos_max,ppos_max,focus_gstart,focus_gstop,...,decay_left,decay_right,baseline,aic,bic,rss,constant_aic,new_contig,new_focus_pstart,new_focus_pstop
0,UG-E_Tororo_gamb_2012_Q4,3RL,37.737016,18868508,1363,0.031600,37.988878,18994439,37.619602,37.812160,...,0.469654,0.300576,0.012386,-15126.530029,-15101.172162,0.003096,-13763.299042,3R,18809802,18906080
1,UG-E_Tororo_gamb_2012_Q4,3RL,57.188070,28594035,3013,0.468600,57.214744,28607372,57.150323,57.235350,...,0.150987,0.189121,0.020351,-10406.921476,-10381.508733,0.189415,-7393.512427,3R,28575162,28617675
2,UG-E_Tororo_gamb_2012_Q4,3RL,64.056450,32028225,628,0.073600,64.099554,32049777,64.022542,64.092043,...,0.135631,0.142371,0.020860,-6095.289986,-6073.731312,0.008492,-5466.805436,3R,32011272,32046022
3,UG-E_Tororo_gamb_2012_Q4,3RL,87.843601,43921801,521,0.148200,87.885688,43942844,87.746791,88.037221,...,0.387240,0.774481,0.044975,-8551.626027,-8527.172981,0.162175,-8030.381883,3R,43873396,44018611
4,ML-2_Kati_gamb_2014_Q3,3RL,38.016228,19008114,1889,0.061632,38.074910,19037455,37.903543,38.072570,...,0.450738,0.225369,0.022625,-14483.216243,-14457.858376,0.005345,-12593.387381,3R,18951772,19036286


In [16]:
#sort the signals dataframe by ascending chromosome name and then position
all_gambiae_signals_df.sort_values(['new_contig', 'new_focus_pstart'], ascending=[True, True], inplace=True, ignore_index=True)   
all_gambiae_signals_df.head()

,cohort_id,contig,gcenter,pcenter,delta_i,stat_max,gpos_max,ppos_max,focus_gstart,focus_gstop,...,decay_left,decay_right,baseline,aic,bic,rss,constant_aic,new_contig,new_focus_pstart,new_focus_pstop
0,CF-BGF_Bangui_gamb_1994_Q1,2RL,119.879728,62827590,1900,0.3830,119.897354,62862842,119.587173,120.194486,...,1.170221,1.259031,0.033095,-13390.353075,-13362.472589,2.029204,-11489.383920,2L,697375,1912001
1,CM-ES_Lom-Et-Djérem_gamb_2009_Q3,2RL,120.590188,64065726,1798,0.5364,119.640736,62349605,119.963278,121.299133,...,2.507640,2.835778,0.074493,-12587.152892,-12559.208743,3.366092,-10788.500102,2L,1449585,2875093
2,CM-ES_Haut-Nyong_gamb_2009_Q3,2RL,120.516091,64028677,2359,0.4056,119.599149,62266431,120.139382,121.009187,...,1.506836,1.972386,0.052667,-13298.956652,-13271.012503,2.347916,-10939.583618,2L,1801792,2730120
3,BF-09_Houet_gamb_2012_Q3,2RL,120.644764,64093013,4164,1.0000,120.414675,63897484,120.318784,120.995416,...,1.303920,1.402611,0.031485,-9366.444904,-9338.500755,17.178618,-5202.186528,2L,2160596,2723235
4,GN-N_Macenta_gamb_2012_Q4,2RL,120.649587,64095425,4326,0.9606,120.596352,64068807,120.341814,120.930505,...,1.231091,1.123670,0.056485,-9687.128655,-9659.184506,14.605178,-5360.945558,2L,2206658,2690779


In [17]:
#need to import the location of genes from the genes file
Agam_genes =pd.read_table('~/lstm_scratch/AI_metaanalysis/AGAM_genes.bed', names=["Chrom", "Start", "End", "GeneID"])

In [18]:
Agam_genes

,Chrom,Start,End,GeneID
0,2R,21850802,21851824,AGAP002473
1,2R,10172461,10177055,AGAP001777
2,2R,10183331,10210950,AGAP001779
3,2R,10178041,10179205,AGAP001778
4,2R,10166570,10167082,AGAP001776
...,...,...,...,...
13079,X,8856702,8905975,AGAP000504
13080,X,23125381,23126267,AGAP001079
13081,X,20029212,20030964,AGAP013541
13082,X,14140099,14140609,AGAP029885


In [19]:
len(Agam_genes)

13084

In [21]:
import allel

In [22]:
#this highlights that the chromosome must also match!
#could make a gene index for each chromosome
gene_idx_list_2L = []
gene_idx_list_2R = []
gene_idx_list_3L = []
gene_idx_list_3R = []
gene_idx_list_X = []
for index, row in Agam_genes.iterrows():
    if Agam_genes['Chrom'].iloc[index] == '2L':
        gene_idx_list_2L.append(allel.SortedIndex(Agam_genes[['Start','End']].iloc[index]))
    elif Agam_genes['Chrom'].iloc[index] == '2R':
        gene_idx_list_2R.append(allel.SortedIndex(Agam_genes[['Start','End']].iloc[index]))
    elif Agam_genes['Chrom'].iloc[index] == '3L':
        gene_idx_list_3L.append(allel.SortedIndex(Agam_genes[['Start','End']].iloc[index]))
    elif Agam_genes['Chrom'].iloc[index] == '3R':
        gene_idx_list_3R.append(allel.SortedIndex(Agam_genes[['Start','End']].iloc[index]))
    elif Agam_genes['Chrom'].iloc[index] == 'X':
        gene_idx_list_X.append(allel.SortedIndex(Agam_genes[['Start','End']].iloc[index]))

        
    

In [23]:
#check that all the genes are included
len(gene_idx_list_2L)+ len(gene_idx_list_2R)+ len(gene_idx_list_3L)+ len(gene_idx_list_3R)+ len(gene_idx_list_X) == len(Agam_genes)

True

In [24]:
gene_idx_lists = [gene_idx_list_2L, gene_idx_list_2R, gene_idx_list_3L, gene_idx_list_3R, gene_idx_list_X]

In [25]:
gene_idx_array_2L = np.array(gene_idx_list_2L)
gene_idx_array_2R = np.array(gene_idx_list_2R)
gene_idx_array_3L = np.array(gene_idx_list_3L)
gene_idx_array_3R = np.array(gene_idx_list_3R)
gene_idx_array_X = np.array(gene_idx_list_X)

In [26]:
#gene start positions are  out by 1 position as bed file has 0 based coordinates
#4e.g. for AGAP004707 the expected starting position is 2358158
Agam_genes[Agam_genes['GeneID'] == 'AGAP004707']

,Chrom,Start,End,GeneID
8316,2L,2358157,2431617,AGAP004707


In [27]:
gene_idx_arrays = [gene_idx_array_2L, gene_idx_array_2R, gene_idx_array_3L, gene_idx_array_3R, gene_idx_array_X]

In [28]:
#same sort of thing for the swept regions on each chromosomal arm - this is for the Ugandan samples only
idx_list_2L = []
idx_list_2R = []
idx_list_3L = []
idx_list_3R = []
idx_list_X = []

for index,row in all_gambiae_signals_df.iterrows():
    if all_gambiae_signals_df['new_contig'].iloc[index]=='2L':
        idx_list_2L.append(allel.SortedIndex(all_gambiae_signals_df[['new_focus_pstart','new_focus_pstop']].iloc[index]))
    elif all_gambiae_signals_df['new_contig'].iloc[index]=='2R':
        idx_list_2R.append(allel.SortedIndex(all_gambiae_signals_df[['new_focus_pstart','new_focus_pstop']].iloc[index]))
    elif all_gambiae_signals_df['new_contig'].iloc[index]=='3L':
        idx_list_3L.append(allel.SortedIndex(all_gambiae_signals_df[['new_focus_pstart','new_focus_pstop']].iloc[index]))
    elif all_gambiae_signals_df['new_contig'].iloc[index]=='3R':
        idx_list_3R.append(allel.SortedIndex(all_gambiae_signals_df[['new_focus_pstart','new_focus_pstop']].iloc[index]))
    elif all_gambiae_signals_df['new_contig'].iloc[index]=='X':
        idx_list_X.append(allel.SortedIndex(all_gambiae_signals_df[['new_focus_pstart','new_focus_pstop']].iloc[index]))
idx_list_2R    

[<SortedIndex shape=(2,) dtype=int64>
 [3027447, 3376144],
 <SortedIndex shape=(2,) dtype=int64>
 [3229249, 3358883],
 <SortedIndex shape=(2,) dtype=int64>
 [3281349, 3421850],
 <SortedIndex shape=(2,) dtype=int64>
 [3305240, 3587651],
 <SortedIndex shape=(2,) dtype=int64>
 [3320515, 3504494],
 <SortedIndex shape=(2,) dtype=int64>
 [3323883, 3391441],
 <SortedIndex shape=(2,) dtype=int64>
 [8601919, 8713989],
 <SortedIndex shape=(2,) dtype=int64>
 [9748581, 9821485],
 <SortedIndex shape=(2,) dtype=int64>
 [12948453, 12974120],
 <SortedIndex shape=(2,) dtype=int64>
 [17311011, 17337528],
 <SortedIndex shape=(2,) dtype=int64>
 [19047751, 19074315],
 <SortedIndex shape=(2,) dtype=int64>
 [19048250, 19073753],
 <SortedIndex shape=(2,) dtype=int64>
 [19116351, 19262502],
 <SortedIndex shape=(2,) dtype=int64>
 [19122783, 19178806],
 <SortedIndex shape=(2,) dtype=int64>
 [19280351, 19416152],
 <SortedIndex shape=(2,) dtype=int64>
 [28373806, 28553041],
 <SortedIndex shape=(2,) dtype=int64>
 [

In [29]:
#check that all 30 regions are still there!
all_idx_lists = [idx_list_2L, idx_list_2R, idx_list_3L, idx_list_3R, idx_list_X]
list_length = 0
for idx_list in all_idx_lists:
    list_length = list_length + len(idx_list)
list_length


222

In [30]:
idx_list_2L[0]

0,1
697375,1912001


In [34]:
#next step to do a similar thing but loop through all the index positions
#this gives all the start positions in the range of the first gene - it is clear that the loc intersection ranges isn't working properly
gene_start_overlapping_2L = []
gene_starts = []
starts = gene_idx_array_2L[:, 0]
ends = gene_idx_array_2L[:, 1]
for idx in idx_list_2L:
    print(idx)
    tmp=[]
    for start, end in zip (starts, ends):
        if idx[0] < start and start < idx[1] and idx[0] < end and end< idx[1]:
            print(start)  
            tmp.append(start)
    gene_start_overlapping_2L.append(tmp)
gene_start_overlapping_2L

[697375, 1912001]
819112
1052035
1133036
1411010
1834185
931775
1315237
742003
1772468
1628981
1594182
1270279
1185939
1760890
1011772
1008859
1527098
[1449585, 2875093]
1834185
2248666
2305656
2482552
2483225
2506571
2752350
2797663
2103173
2358157
2567154
2714471
1772468
1628981
1594182
2749617
1967082
2074971
2803038
2823489
2487697
1923903
2726409
2253669
2615612
1973600
2756403
1760890
2271009
2013633
2812194
2843748
2471588
2775210
2864708
2751785
2558996
2650810
2623950
1527098
2642892
2686883
[1801792, 2730120]
1834185
2248666
2305656
2482552
2483225
2506571
2103173
2358157
2567154
2714471
1967082
2074971
2487697
1923903
2253669
2615612
1973600
2271009
2013633
2471588
2558996
2650810
2623950
2642892
2686883
[2160596, 2723235]
2248666
2305656
2482552
2483225
2506571
2358157
2567154
2714471
2487697
2253669
2615612
2271009
2471588
2558996
2650810
2623950
2642892
2686883
[2206658, 2690779]
2248666
2305656
2482552
2483225
2506571
2358157
2567154
2487697
2253669
2615612
2271009
24715

[32360667, 32398732]
32370952
32364920
[33127272, 33834378]
33166022
33168444
33171935
33177520
33205176
33776281
33786624
33825710
33172674
33780245
33767544
33288832
33478485
33210937
33567017
33292932
[33956335, 34133685]
34064108
34077591
34087429
34107954
34111265
34115121
34117677
34120725
34122194
34124498
34125950
34128578
34130954
34133345
34100050
34057953
34076527
33963336
34113356
34093934
34049559
33986816
34022245
[33982641, 34228452]
34064108
34077591
34087429
34107954
34111265
34115121
34117677
34120725
34122194
34124498
34125950
34128578
34130954
34133345
34137004
34141797
34143240
34143855
34145657
34146267
34148654
34152344
34168870
34197255
34203892
34207127
34220001
34221212
34225626
34148065
34158498
34100050
34057953
34170871
34076527
34113356
34093934
34049559
33986816
34151631
34150792
34150049
34022245
[34060553, 34168419]
34064108
34077591
34087429
34107954
34111265
34115121
34117677
34120725
34122194
34124498
34125950
34128578
34130954
34133345
34137004
3414

[[819112,
  1052035,
  1133036,
  1411010,
  1834185,
  931775,
  1315237,
  742003,
  1772468,
  1628981,
  1594182,
  1270279,
  1185939,
  1760890,
  1011772,
  1008859,
  1527098],
 [1834185,
  2248666,
  2305656,
  2482552,
  2483225,
  2506571,
  2752350,
  2797663,
  2103173,
  2358157,
  2567154,
  2714471,
  1772468,
  1628981,
  1594182,
  2749617,
  1967082,
  2074971,
  2803038,
  2823489,
  2487697,
  1923903,
  2726409,
  2253669,
  2615612,
  1973600,
  2756403,
  1760890,
  2271009,
  2013633,
  2812194,
  2843748,
  2471588,
  2775210,
  2864708,
  2751785,
  2558996,
  2650810,
  2623950,
  1527098,
  2642892,
  2686883],
 [1834185,
  2248666,
  2305656,
  2482552,
  2483225,
  2506571,
  2103173,
  2358157,
  2567154,
  2714471,
  1967082,
  2074971,
  2487697,
  1923903,
  2253669,
  2615612,
  1973600,
  2271009,
  2013633,
  2471588,
  2558996,
  2650810,
  2623950,
  2642892,
  2686883],
 [2248666,
  2305656,
  2482552,
  2483225,
  2506571,
  2358157,
  2567154,

In [35]:
len(gene_start_overlapping_2L[1])

42

In [36]:
#next step to do a similar thing but loop through all the index positions
#this gives all the start positions in the range of the first gene - it is clear that the loc intersection ranges isn't working properly
gene_start_overlapping_2R = []
gene_starts = []
starts = gene_idx_array_2R[:, 0]
ends = gene_idx_array_2R[:, 1]
for idx in idx_list_2R:
    print(idx)
    tmp=[]
    for start, end in zip (starts, ends):
        if idx[0] < start and start < idx[1] and idx[0] < end and end< idx[1]:
            print(start)  
            tmp.append(start)
    gene_start_overlapping_2R.append(tmp)
gene_start_overlapping_2R

[3027447, 3376144]
3039805
3094227
3124553
3226132
3262687
3322959
3324750
3362646
3330416
3286020
3260239
3253874
3251612
3247691
3312794
3308576
3331549
3319080
[3229249, 3358883]
3262687
3322959
3324750
3330416
3286020
3260239
3253874
3251612
3247691
3312794
3308576
3331549
3319080
[3281349, 3421850]
3322959
3324750
3362646
3380536
3382257
3394541
3330416
3286020
3358518
3312794
3308576
3331549
3319080
3384309
[3305240, 3587651]
3322959
3324750
3362646
3380536
3382257
3435806
3434158
3432983
3430764
3394541
3330416
3501595
3546130
3571834
3580983
3568117
3581704
3358518
3312794
3503881
3578078
3308576
3483098
3331549
3396237
3319080
3384309
[3320515, 3504494]
3322959
3324750
3362646
3380536
3382257
3435806
3434158
3432983
3430764
3394541
3330416
3501595
3358518
3483098
3331549
3396237
3384309
[3323883, 3391441]
3324750
3362646
3380536
3382257
3330416
3358518
3331549
[8601919, 8713989]
8673914
[9748581, 9821485]
[12948453, 12974120]
12971257
[17311011, 17337528]
17331870
17333034
173

[[3039805,
  3094227,
  3124553,
  3226132,
  3262687,
  3322959,
  3324750,
  3362646,
  3330416,
  3286020,
  3260239,
  3253874,
  3251612,
  3247691,
  3312794,
  3308576,
  3331549,
  3319080],
 [3262687,
  3322959,
  3324750,
  3330416,
  3286020,
  3260239,
  3253874,
  3251612,
  3247691,
  3312794,
  3308576,
  3331549,
  3319080],
 [3322959,
  3324750,
  3362646,
  3380536,
  3382257,
  3394541,
  3330416,
  3286020,
  3358518,
  3312794,
  3308576,
  3331549,
  3319080,
  3384309],
 [3322959,
  3324750,
  3362646,
  3380536,
  3382257,
  3435806,
  3434158,
  3432983,
  3430764,
  3394541,
  3330416,
  3501595,
  3546130,
  3571834,
  3580983,
  3568117,
  3581704,
  3358518,
  3312794,
  3503881,
  3578078,
  3308576,
  3483098,
  3331549,
  3396237,
  3319080,
  3384309],
 [3322959,
  3324750,
  3362646,
  3380536,
  3382257,
  3435806,
  3434158,
  3432983,
  3430764,
  3394541,
  3330416,
  3501595,
  3358518,
  3483098,
  3331549,
  3396237,
  3384309],
 [3324750, 33626

In [37]:
#next step to do a similar thing but loop through all the index positions
#this gives all the start positions in the range of the first gene - it is clear that the loc intersection ranges isn't working properly
gene_start_overlapping_3L = []
gene_starts = []
starts = gene_idx_array_3L[:, 0]
ends = gene_idx_array_3L[:, 1]
for idx in idx_list_3L:
    print(idx)
    tmp=[]
    for start, end in zip (starts, ends):
        if idx[0] < start and start < idx[1] and idx[0] < end and end< idx[1]:
            print(start)  
            tmp.append(start)
    gene_start_overlapping_3L.append(tmp)
gene_start_overlapping_3L

[14396741, 14422876]
14406218
14401561
[14399725, 14426242]
14424684
14406218
14401561
[14400468, 14426984]
14424684
14406218
14401561
[14400485, 14430234]
14426228
14424684
14406218
14401561
[14405349, 14430565]
14426228
14424684
14406218
[14408397, 14434719]
14426228
14424684
14429574
[14410259, 14435825]
14426228
14433124
14424684
14429574
[20154963, 20330956]
20278163
20314437
20313999
20297731
20311292
20294604
20250050
[23329454, 23372976]
23351999
[23338753, 23374290]
23351999
[23343258, 23379220]
23351999
[23352858, 23387253]
[25058896, 25153646]
25091435
[26891240, 26921031]
26892949
26911904
26900212
26904463
[26894004, 26920222]
26911904
26900212
26904463
[26913168, 26942248]
26920631
26925425
[26913496, 26940429]
26920631
26925425
[26914445, 26942435]
26920631
26925425
[26915645, 26940804]
26920631
26925425
[26917165, 26944829]
26941807
26920631
26925425
[26917819, 26944714]
26941807
26920631
26925425
[26918812, 26945595]
26941807
26920631
26925425
[26932933, 26962074]
2694

[[14406218, 14401561],
 [14424684, 14406218, 14401561],
 [14424684, 14406218, 14401561],
 [14426228, 14424684, 14406218, 14401561],
 [14426228, 14424684, 14406218],
 [14426228, 14424684, 14429574],
 [14426228, 14433124, 14424684, 14429574],
 [20278163, 20314437, 20313999, 20297731, 20311292, 20294604, 20250050],
 [23351999],
 [23351999],
 [23351999],
 [],
 [25091435],
 [26892949, 26911904, 26900212, 26904463],
 [26911904, 26900212, 26904463],
 [26920631, 26925425],
 [26920631, 26925425],
 [26920631, 26925425],
 [26920631, 26925425],
 [26941807, 26920631, 26925425],
 [26941807, 26920631, 26925425],
 [26941807, 26920631, 26925425],
 [26946961, 26951275, 26944019, 26941807],
 [38123321, 38121355],
 [38253903, 38263672, 38269560, 38273256]]

In [38]:
gene_idx_array_2L

array([[  157347,   186936],
       [  203778,   205293],
       [  207893,   210460],
       ...,
       [  476275,   484647],
       [37674152, 37700872],
       [45638400, 45640120]])

In [39]:
#next step to do a similar thing but loop through all the index positions
#this gives all the start positions in the range of the first gene - it is clear that the loc intersection ranges isn't working properly
gene_start_overlapping_3R = []
gene_starts = []
starts = gene_idx_array_3R[:, 0]
ends = gene_idx_array_3R[:, 1]
for idx in idx_list_3R:
    print(idx)
    tmp=[]
    for start, end in zip (starts, ends):
        if idx[0] < start and start < idx[1] and idx[0] < end and end< idx[1]:
            print(start)  
            tmp.append(start)
    gene_start_overlapping_3R.append(tmp)
gene_start_overlapping_3R

[4415236, 4463288]
[4457658, 4516262]
4492631
4494334
[4478105, 4592047]
4511375
4492631
4555078
4551712
4494334
4523692
[4588981, 4746324]
[6913813, 6945990]
6933858
6931810
6915901
6927994
6922016
6928857
[6926043, 6953600]
6933858
6931810
6927994
6928857
[18686589, 18755971]
[18713137, 18781866]
[18781614, 18835752]
18814968
[18790137, 18848073]
18814968
[18794743, 18874454]
18814968
[18794772, 18873945]
18814968
[18798405, 18844608]
18814968
[18806491, 18847901]
18814968
[18809802, 18906080]
18814968
18903440
[18810481, 18852870]
18814968
[18819620, 18896796]
[18951772, 19036286]
[28382487, 28557098]
28493612
[28456878, 28547727]
28493612
[28469398, 28618601]
28607055
28571276
28593713
28601367
28595947
28598870
28594992
28573572
28597651
28576091
28568854
28493612
28590242
28611019
28600346
[28498960, 28585578]
28571276
28573572
28568854
[28502027, 28596476]
28571276
28593713
28594992
28573572
28576091
28568854
28590242
[28502322, 28599923]
28571276
28593713
28595947
28598870
2859

[[],
 [4492631, 4494334],
 [4511375, 4492631, 4555078, 4551712, 4494334, 4523692],
 [],
 [6933858, 6931810, 6915901, 6927994, 6922016, 6928857],
 [6933858, 6931810, 6927994, 6928857],
 [],
 [],
 [18814968],
 [18814968],
 [18814968],
 [18814968],
 [18814968],
 [18814968],
 [18814968, 18903440],
 [18814968],
 [],
 [],
 [28493612],
 [28493612],
 [28607055,
  28571276,
  28593713,
  28601367,
  28595947,
  28598870,
  28594992,
  28573572,
  28597651,
  28576091,
  28568854,
  28493612,
  28590242,
  28611019,
  28600346],
 [28571276, 28573572, 28568854],
 [28571276, 28593713, 28594992, 28573572, 28576091, 28568854, 28590242],
 [28571276,
  28593713,
  28595947,
  28598870,
  28594992,
  28573572,
  28597651,
  28576091,
  28568854,
  28590242],
 [28571276, 28573572, 28568854],
 [28607055,
  28571276,
  28593713,
  28601367,
  28595947,
  28598870,
  28594992,
  28573572,
  28597651,
  28576091,
  28568854,
  28590242,
  28611019,
  28612695,
  28600346],
 [28571276,
  28593713,
  28601367

In [40]:
#next step to do a similar thing but loop through all the index positions
#this gives all the start positions in the range of the first gene - it is clear that the loc intersection ranges isn't working properly
gene_start_overlapping_X = []
gene_starts = []
starts = gene_idx_array_X[:, 0]
ends = gene_idx_array_X[:, 1]
for idx in idx_list_X:
    print(idx)
    tmp=[]
    for start, end in zip (starts, ends):
        if idx[0] < start and start < idx[1] and idx[0] < end and end< idx[1]:
            print(start)  
            tmp.append(start)
    gene_start_overlapping_X.append(tmp)
gene_start_overlapping_X

[9141000, 9170104]
[9157421, 9361426]
9297186
9215206
9290785
[9210452, 9235831]
[9210974, 9246214]
[9211001, 9246187]
[9214973, 9241489]
[14999682, 15205157]
15003957
15007940
15013480
15053436
15160655
15029188
15188579
15189808
15195113
15085094
15027285
[15057208, 15403805]
15240571
15160655
15252294
15327410
15375385
15378923
15383616
15304984
15335807
15281335
15348089
15341868
15319509
15365553
15355730
15188579
15189808
15195113
15085094
[15066863, 15250413]
15240571
15160655
15188579
15189808
15195113
15085094
[15077956, 15516679]
15240571
15160655
15252294
15327410
15375385
15378923
15414171
15383616
15304984
15468832
15335807
15281335
15430355
15348089
15341868
15319509
15365553
15438660
15355730
15188579
15189808
15195113
15085094
[15152890, 15305608]
15240571
15160655
15252294
15281335
15188579
15189808
15195113
[15161317, 15316825]
15240571
15252294
15281335
15188579
15189808
15195113
[15179345, 15330498]
15240571
15252294
15304984
15281335
15319509
15188579
15189808
1519

[[],
 [9297186, 9215206, 9290785],
 [],
 [],
 [],
 [],
 [15003957,
  15007940,
  15013480,
  15053436,
  15160655,
  15029188,
  15188579,
  15189808,
  15195113,
  15085094,
  15027285],
 [15240571,
  15160655,
  15252294,
  15327410,
  15375385,
  15378923,
  15383616,
  15304984,
  15335807,
  15281335,
  15348089,
  15341868,
  15319509,
  15365553,
  15355730,
  15188579,
  15189808,
  15195113,
  15085094],
 [15240571, 15160655, 15188579, 15189808, 15195113, 15085094],
 [15240571,
  15160655,
  15252294,
  15327410,
  15375385,
  15378923,
  15414171,
  15383616,
  15304984,
  15468832,
  15335807,
  15281335,
  15430355,
  15348089,
  15341868,
  15319509,
  15365553,
  15438660,
  15355730,
  15188579,
  15189808,
  15195113,
  15085094],
 [15240571, 15160655, 15252294, 15281335, 15188579, 15189808, 15195113],
 [15240571, 15252294, 15281335, 15188579, 15189808, 15195113],
 [15240571,
  15252294,
  15304984,
  15281335,
  15319509,
  15188579,
  15189808,
  15195113],
 [15240571

In [41]:
#to convert this to a list of genes, look up the gene start positions in the table
genes_overlapping_2L = []
for i in range(len(gene_start_overlapping_2L)):
    gene_list = Agam_genes[(Agam_genes['Start'].isin(gene_start_overlapping_2L[i]))]['GeneID'].tolist()
    genes_overlapping_2L.append(gene_list)
genes_overlapping_2L


[['AGAP004687',
  'AGAP004689',
  'AGAP004690',
  'AGAP004692',
  'AGAP004695',
  'AGAP028432',
  'AGAP028433',
  'AGAP004686',
  'AGAP029368',
  'AGAP028434',
  'AGAP004694',
  'AGAP004691',
  'AGAP029344',
  'AGAP028435',
  'AGAP029795',
  'AGAP029798',
  'AGAP004693'],
 ['AGAP004695',
  'AGAP004703',
  'AGAP004706',
  'AGAP004709',
  'AGAP004710',
  'AGAP004712',
  'AGAP004722',
  'AGAP004725',
  'AGAP028436',
  'AGAP004707',
  'AGAP004714',
  'AGAP004719',
  'AGAP029368',
  'AGAP028434',
  'AGAP004694',
  'AGAP029687',
  'AGAP004698',
  'AGAP029667',
  'AGAP004726',
  'AGAP004728',
  'AGAP004711',
  'AGAP004696',
  'AGAP004720',
  'AGAP004704',
  'AGAP004715',
  'AGAP004699',
  'AGAP004723',
  'AGAP028435',
  'AGAP028437',
  'AGAP004700',
  'AGAP004727',
  'AGAP004729',
  'AGAP004708',
  'AGAP004724',
  'AGAP004730',
  'AGAP029688',
  'AGAP004713',
  'AGAP029796',
  'AGAP004716',
  'AGAP004693',
  'AGAP004717',
  'AGAP004718'],
 ['AGAP004695',
  'AGAP004703',
  'AGAP004706',
  'AGA

In [42]:
genes_overlapping_2L[3]

['AGAP004703',
 'AGAP004706',
 'AGAP004709',
 'AGAP004710',
 'AGAP004712',
 'AGAP004707',
 'AGAP004714',
 'AGAP004719',
 'AGAP004711',
 'AGAP004704',
 'AGAP004715',
 'AGAP028437',
 'AGAP004708',
 'AGAP004713',
 'AGAP029796',
 'AGAP004716',
 'AGAP004717',
 'AGAP004718']

In [43]:
for item in genes_overlapping_X:
    if 'AGAP000818 ' in item:
        print(item,"includes 'AGAP000818'")
    else:
        print('AGAP000818  not present')

NameError: name 'genes_overlapping_X' is not defined

In [44]:
#to convert this to a list of genes, look up the gene start positions in the table
genes_overlapping_2R = []
for i in range(len(gene_start_overlapping_2R)):
    gene_list = Agam_genes[(Agam_genes['Start'].isin(gene_start_overlapping_2R[i]))]['GeneID'].tolist()
    genes_overlapping_2R.append(gene_list)
genes_overlapping_2R

[['AGAP001328',
  'AGAP001329',
  'AGAP001330',
  'AGAP001331',
  'AGAP001335',
  'AGAP001340',
  'AGAP001341',
  'AGAP001345',
  'AGAP001342',
  'AGAP001336',
  'AGAP001334',
  'AGAP001333',
  'AGAP001332',
  'AGAP028467',
  'AGAP001338',
  'AGAP001337',
  'AGAP001343',
  'AGAP001339'],
 ['AGAP001335',
  'AGAP001340',
  'AGAP001341',
  'AGAP001342',
  'AGAP001336',
  'AGAP001334',
  'AGAP001333',
  'AGAP001332',
  'AGAP028467',
  'AGAP001338',
  'AGAP001337',
  'AGAP001343',
  'AGAP001339'],
 ['AGAP001340',
  'AGAP001341',
  'AGAP001345',
  'AGAP001346',
  'AGAP001347',
  'AGAP001349',
  'AGAP001342',
  'AGAP001336',
  'AGAP001344',
  'AGAP001338',
  'AGAP001337',
  'AGAP001343',
  'AGAP001339',
  'AGAP001348'],
 ['AGAP001340',
  'AGAP001341',
  'AGAP001345',
  'AGAP001346',
  'AGAP001347',
  'AGAP001355',
  'AGAP001354',
  'AGAP001353',
  'AGAP001352',
  'AGAP001349',
  'AGAP001342',
  'AGAP001357',
  'AGAP001360',
  'AGAP001362',
  'AGAP001364',
  'AGAP001361',
  'AGAP001365',
  'AG

In [45]:
#to convert this to a list of genes, look up the gene start positions in the table
genes_overlapping_3L = []
for i in range(len(gene_start_overlapping_3L)):
    gene_list = Agam_genes[(Agam_genes['Start'].isin(gene_start_overlapping_3L[i]))]['GeneID'].tolist()
    genes_overlapping_3L.append(gene_list)
genes_overlapping_3L

[['AGAP029593', 'AGAP010975'],
 ['AGAP010978', 'AGAP029593', 'AGAP010975'],
 ['AGAP010978', 'AGAP029593', 'AGAP010975'],
 ['AGAP029729', 'AGAP010978', 'AGAP029593', 'AGAP010975'],
 ['AGAP029729', 'AGAP010978', 'AGAP029593'],
 ['AGAP029729', 'AGAP010978', 'AGAP029730'],
 ['AGAP029729', 'AGAP029727', 'AGAP010978', 'AGAP029730'],
 ['AGAP011256',
  'AGAP011261',
  'AGAP011260',
  'AGAP011259',
  'AGAP029736',
  'AGAP029737',
  'AGAP011255'],
 ['AGAP011390'],
 ['AGAP011390'],
 ['AGAP011390'],
 [],
 ['AGAP011480'],
 ['AGAP011534', 'AGAP029751', 'AGAP011535', 'AGAP011536'],
 ['AGAP029751', 'AGAP011535', 'AGAP011536'],
 ['AGAP028430', 'AGAP029752'],
 ['AGAP028430', 'AGAP029752'],
 ['AGAP028430', 'AGAP029752'],
 ['AGAP028430', 'AGAP029752'],
 ['AGAP011538', 'AGAP028430', 'AGAP029752'],
 ['AGAP011538', 'AGAP028430', 'AGAP029752'],
 ['AGAP011538', 'AGAP028430', 'AGAP029752'],
 ['AGAP011540', 'AGAP011541', 'AGAP011539', 'AGAP011538'],
 ['AGAP012160', 'AGAP012159'],
 ['AGAP012167', 'AGAP012168', 'A

In [46]:
#to convert this to a list of genes, look up the gene start positions in the table
genes_overlapping_3R = []
for i in range(len(gene_start_overlapping_3R)):
    gene_list = Agam_genes[(Agam_genes['Start'].isin(gene_start_overlapping_3R[i]))]['GeneID'].tolist()
    genes_overlapping_3R.append(gene_list)
genes_overlapping_3R

[[],
 ['AGAP013728', 'AGAP008026'],
 ['AGAP008027',
  'AGAP013728',
  'AGAP008030',
  'AGAP008029',
  'AGAP008026',
  'AGAP008028'],
 [],
 ['AGAP008214',
  'AGAP008213',
  'AGAP008209',
  'AGAP008211',
  'AGAP008210',
  'AGAP008212'],
 ['AGAP008214', 'AGAP008213', 'AGAP008211', 'AGAP008212'],
 [],
 [],
 ['AGAP008824'],
 ['AGAP008824'],
 ['AGAP008824'],
 ['AGAP008824'],
 ['AGAP008824'],
 ['AGAP008824'],
 ['AGAP008824', 'AGAP008825'],
 ['AGAP008824'],
 [],
 [],
 ['AGAP009184'],
 ['AGAP009184'],
 ['AGAP009198',
  'AGAP009187',
  'AGAP009191',
  'AGAP009197',
  'AGAP009193',
  'AGAP009195',
  'AGAP009192',
  'AGAP009188',
  'AGAP009194',
  'AGAP009189',
  'AGAP009185',
  'AGAP009184',
  'AGAP009190',
  'AGAP009199',
  'AGAP009196'],
 ['AGAP009187', 'AGAP009188', 'AGAP009185'],
 ['AGAP009187',
  'AGAP009191',
  'AGAP009192',
  'AGAP009188',
  'AGAP009189',
  'AGAP009185',
  'AGAP009190'],
 ['AGAP009187',
  'AGAP009191',
  'AGAP009193',
  'AGAP009195',
  'AGAP009192',
  'AGAP009188',
  'AGAP

In [47]:
#to convert this to a list of genes, look up the gene start positions in the table
genes_overlapping_X = []
for i in range(len(gene_start_overlapping_X)):
    gene_list = Agam_genes[(Agam_genes['Start'].isin(gene_start_overlapping_X[i]))]['GeneID'].tolist()
    genes_overlapping_X.append(gene_list)
genes_overlapping_X

[[],
 ['AGAP000521', 'AGAP000519', 'AGAP000520'],
 [],
 [],
 [],
 [],
 ['AGAP000817',
  'AGAP000816',
  'AGAP013474',
  'AGAP013173',
  'AGAP012997',
  'AGAP029887',
  'AGAP029884',
  'AGAP029890',
  'AGAP029888',
  'AGAP013424',
  'AGAP029892'],
 ['AGAP000818',
  'AGAP012997',
  'AGAP000819',
  'AGAP000823',
  'AGAP000831',
  'AGAP000832',
  'AGAP000833',
  'AGAP000821',
  'AGAP000824',
  'AGAP000820',
  'AGAP000826',
  'AGAP000825',
  'AGAP000822',
  'AGAP000830',
  'AGAP000829',
  'AGAP029884',
  'AGAP029890',
  'AGAP029888',
  'AGAP013424'],
 ['AGAP000818',
  'AGAP012997',
  'AGAP029884',
  'AGAP029890',
  'AGAP029888',
  'AGAP013424'],
 ['AGAP000818',
  'AGAP012997',
  'AGAP000819',
  'AGAP000823',
  'AGAP000831',
  'AGAP000832',
  'AGAP000834',
  'AGAP000833',
  'AGAP000821',
  'AGAP013506',
  'AGAP000824',
  'AGAP000820',
  'AGAP000835',
  'AGAP000826',
  'AGAP000825',
  'AGAP000822',
  'AGAP000830',
  'AGAP028655',
  'AGAP000829',
  'AGAP029884',
  'AGAP029890',
  'AGAP029888',

In [50]:
#can now put together the gene lists with the table of signals
#first get all the gene lists into one big list
all_gambiae_gene_lists = [genes_overlapping_2L, genes_overlapping_2R, genes_overlapping_3L, genes_overlapping_3R, genes_overlapping_X]
all_list = []
for gene_list in all_gambiae_gene_lists:
    for small_list in gene_list:
        all_list.append(small_list)
len(all_list)

222

In [51]:
#use join to put together the genes in each region to comma separaed
overlapping_genes_for_gambiae_df = []
for thing in all_list:
        gene_string = ' '.join(thing)
        overlapping_genes_for_gambiae_df.append(str(gene_string))

len(overlapping_genes_for_gambiae_df)

222

In [52]:
#then append this to the UG signals dataframe
#now need to append this to a dataframe of regions
#make a new dataframe and add this in - it seems to prefer being concatenated on the 1st axis
gambiae_sweeps_overlapping_genes_df = all_gambiae_signals_df[['cohort_id','new_contig','new_focus_pstart','new_focus_pstop']]
#regions_and_overlapping_genes_df['overlapping_genes'] = overlapping_genes_for_df
gambiae_sweeps_overlapping_genes_df = pd.concat((gambiae_sweeps_overlapping_genes_df, pd.DataFrame(overlapping_genes_for_UG_df)), axis =1, ignore_index=True)
gambiae_sweeps_overlapping_genes_df.columns =['cohort_id','new_contig','new_focus_pstart','new_focus_pstop','overlapping_genes']
gambiae_sweeps_overlapping_genes_df

,cohort_id,new_contig,new_focus_pstart,new_focus_pstop,overlapping_genes
0,CF-BGF_Bangui_gamb_1994_Q1,2L,697375,1912001,AGAP004687 AGAP004689 AGAP004690 AGAP004692 AG...
1,CM-ES_Lom-Et-Djérem_gamb_2009_Q3,2L,1449585,2875093,AGAP004695 AGAP004703 AGAP004706 AGAP004709 AG...
2,CM-ES_Haut-Nyong_gamb_2009_Q3,2L,1801792,2730120,AGAP004695 AGAP004703 AGAP004706 AGAP004709 AG...
3,BF-09_Houet_gamb_2012_Q3,2L,2160596,2723235,AGAP004703 AGAP004706 AGAP004709 AGAP004710 AG...
4,GN-N_Macenta_gamb_2012_Q4,2L,2206658,2690779,AGAP004703 AGAP004706 AGAP004709 AGAP004710 AG...
...,...,...,...,...,...
217,CM-ES_Lom-Et-Djérem_gamb_2009_Q3,X,15421109,15472386,AGAP000835 AGAP028655
218,CF-BGF_Bangui_gamb_1994_Q1,X,15427476,15475943,AGAP000835 AGAP028655
219,BF-09_Houet_gamb_2012_Q3,X,20499760,23324523,AGAP001051 AGAP001052 AGAP001053 AGAP001057 AG...
220,ML-3_Yanfolila_gamb_2012_Q4,X,20552409,23054237,AGAP001051 AGAP001052 AGAP001053 AGAP001057 AG...


In [53]:
#now export this dataframe to a csv
gambiae_sweeps_overlapping_genes_df.to_csv(path_or_buf='gambiae_sweeps_overlapping_genes.csv', sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, mode='w', encoding=None, compression='infer', quoting=None, quotechar='"', doublequote=True)


In [141]:
#attemtping to put into a loop - not gov very far yet!
for contig in contigs:
    globals()[f'gene_starts_overlapping_{contig}']= []
    gene_starts = []
    globals()[f'starts'] = f'gene_idx_array_{contig}[:,0]'
    globals()[f'ends'] = f'gene_idx_array_{contig}[:,1]'
    for idx in all_idx_lists:
        print(idx)
        tmp=[]
        for start, end in zip (starts, ends):
            if idx[0] < start and start < idx[1] and idx[0] < end and end< idx[1]:
                print(start)  
                tmp.append(start)
    gene_start_overlapping_test.append(tmp)
    
 

    #ends = f'gene_idx_array_{contig}[:, 1]'

print(starts)

[<SortedIndex shape=(2,) dtype=int64>
[4008346, 4407263], <SortedIndex shape=(2,) dtype=int64>
[4196125, 4551981], <SortedIndex shape=(2,) dtype=int64>
[9680736, 10387842], <SortedIndex shape=(2,) dtype=int64>
[14597492, 14646139], <SortedIndex shape=(2,) dtype=int64>
[17914314, 17946941], <SortedIndex shape=(2,) dtype=int64>
[20699187, 20748245], <SortedIndex shape=(2,) dtype=int64>
[20717188, 20789534], <SortedIndex shape=(2,) dtype=int64>
[24036118, 24061129], <SortedIndex shape=(2,) dtype=int64>
[35368987, 35546337], <SortedIndex shape=(2,) dtype=int64>
[35473205, 35581071], <SortedIndex shape=(2,) dtype=int64>
[43550978, 43752190], <SortedIndex shape=(2,) dtype=int64>
[44164910, 44220273]]


TypeError: '<' not supported between instances of 'numpy.ndarray' and 'str'

In [925]:
#now need to append this to a dataframe of regions
#make a new dataframe and add this in - it seems to prefer being concatenated on the 1st axis
regions_and_overlapping_genes_df = all_UG_signals_df[['cohort_id','new_contig','new_focus_pstart','new_focus_pstop']]
#regions_and_overlapping_genes_df['overlapping_genes'] = overlapping_genes_for_df
regions_and_overlapping_genes_df1 = pd.concat((regions_and_overlapping_genes_df , pd.DataFrame(overlapping_genes_for_df)), axis =1, ignore_index=True)
regions_and_overlapping_genes_df1

,0,1,2,3,4
0,UG-W_Kanungu_gamb_2012_Q4,2L,4008346.0,4407263.0,AGAP029667
1,UG-E_Tororo_gamb_2012_Q4,2L,4196125.0,4551981.0,AGAP004835
2,UG-E_Tororo_gamb_2012_Q4,2L,9680736.0,10387842.0,"AGAP004827,AGAP004767"
3,UG-W_Kanungu_gamb_2012_Q4,2L,14597492.0,14646139.0,AGAP004827
4,UG-W_Kanungu_gamb_2012_Q4,2L,17914314.0,17946941.0,AGAP004795
...,...,...,...,...,...
217,NaN,NaN,NaN,NaN,"AGAP000834,AGAP013506"
218,NaN,NaN,NaN,NaN,AGAP013506
219,NaN,NaN,NaN,NaN,0
220,NaN,NaN,NaN,NaN,0


In [ ]:
#check does this make sense? - suspicious how few genes in each range. Try combining the chromosoems the other way round

In [907]:
#Make idx lists for each swept region in all gambiae sweeps
idx_list_2L = []
idx_list_2R = []
idx_list_3L = []
idx_list_3R = []
idx_list_X = []

for index,row in all_gambiae_signals_df.iterrows():
    if all_gambiae_signals_df['new_contig'].iloc[index]=='2L':
        idx_list_2L.append(allel.SortedIndex(all_gambiae_signals_df[['new_focus_pstart','new_focus_pstop']].iloc[index]))
    elif all_gambiae_signals_df['new_contig'].iloc[index]=='2R':
        idx_list_2R.append(allel.SortedIndex(all_gambiae_signals_df[['new_focus_pstart','new_focus_pstop']].iloc[index]))
    elif all_gambiae_signals_df['new_contig'].iloc[index]=='3L':
        idx_list_3L.append(allel.SortedIndex(all_gambiae_signals_df[['new_focus_pstart','new_focus_pstop']].iloc[index]))
    elif all_gambiae_signals_df['new_contig'].iloc[index]=='3R':
        idx_list_3R.append(allel.SortedIndex(all_gambiae_signals_df[['new_focus_pstart','new_focus_pstop']].iloc[index]))
    elif all_gambiae_signals_df['new_contig'].iloc[index]=='X':
        idx_list_X.append(allel.SortedIndex(all_gambiae_signals_df[['new_focus_pstart','new_focus_pstop']].iloc[index]))
idx_list_2R    

[<SortedIndex shape=(2,) dtype=int64>
 [3027447, 3376144],
 <SortedIndex shape=(2,) dtype=int64>
 [3229249, 3358883],
 <SortedIndex shape=(2,) dtype=int64>
 [3281349, 3421850],
 <SortedIndex shape=(2,) dtype=int64>
 [3305240, 3587651],
 <SortedIndex shape=(2,) dtype=int64>
 [3320515, 3504494],
 <SortedIndex shape=(2,) dtype=int64>
 [3323883, 3391441],
 <SortedIndex shape=(2,) dtype=int64>
 [8601919, 8713989],
 <SortedIndex shape=(2,) dtype=int64>
 [9748581, 9821485],
 <SortedIndex shape=(2,) dtype=int64>
 [12948453, 12974120],
 <SortedIndex shape=(2,) dtype=int64>
 [17311011, 17337528],
 <SortedIndex shape=(2,) dtype=int64>
 [19047751, 19074315],
 <SortedIndex shape=(2,) dtype=int64>
 [19048250, 19073753],
 <SortedIndex shape=(2,) dtype=int64>
 [19116351, 19262502],
 <SortedIndex shape=(2,) dtype=int64>
 [19122783, 19178806],
 <SortedIndex shape=(2,) dtype=int64>
 [19280351, 19416152],
 <SortedIndex shape=(2,) dtype=int64>
 [28373806, 28553041],
 <SortedIndex shape=(2,) dtype=int64>
 [

In [908]:
#check that all regions are still there (222 for all gambiae)
all_idx_lists = [idx_list_2L, idx_list_2R, idx_list_3L, idx_list_3R, idx_list_X]
list_length = 0
for idx_list in all_idx_lists:
    list_length = list_length + len(idx_list)
list_length

222

In [909]:
#may not be most efficient way to check which ranges overlap
#but it seems to work
overlapping_genes = []
for i in range(len(gene_idx_arrays)):
    gene_idx_array= gene_idx_arrays[i]
    idx_list = all_idx_lists[i]
    if idx_list == idx_list_2L:
        for region in idx_list:
        #print(region)
            #print(gene_idx_array_2L[:, 0])
            loc, loc_ranges = region.locate_intersection_ranges(gene_idx_array_2L[:, 0], gene_idx_array_2L[:, 1])
            overlapping_genes.append(Agam_genes.loc[Agam_genes['Start'].isin(gene_idx_array_2L[loc_ranges][:, 0])]['GeneID'])
    elif idx_list == idx_list_2R:
        for region in idx_list:
            #print(gene_idx_array_2R[:, 0])
            loc, loc_ranges = region.locate_intersection_ranges(gene_idx_array_2R[:, 0], gene_idx_array_2R[:, 1])
            overlapping_genes.append(Agam_genes.loc[Agam_genes['Start'].isin(gene_idx_array_2R[loc_ranges][:, 0])]['GeneID'])
    elif idx_list == idx_list_3L:
        for region in idx_list:
            #print(gene_idx_array_3L[:, 0])
            loc, loc_ranges = region.locate_intersection_ranges(gene_idx_array_3L[:, 0], gene_idx_array_3L[:, 1])
            overlapping_genes.append(Agam_genes.loc[Agam_genes['Start'].isin(gene_idx_array_3L[loc_ranges][:, 0])]['GeneID'])
    elif idx_list == idx_list_3R:
        for region in idx_list:
            #print(gene_idx_array_3R[:, 0])
            loc, loc_ranges = region.locate_intersection_ranges(gene_idx_array_3R[:, 0], gene_idx_array_3R[:, 1])
            overlapping_genes.append(Agam_genes.loc[Agam_genes['Start'].isin(gene_idx_array_3R[loc_ranges][:, 0])]['GeneID'])
    elif idx_list == idx_list_X:
        for region in idx_list:
            #print(gene_idx_array_X[:, 0])
            loc, loc_ranges = region.locate_intersection_ranges(gene_idx_array_X[:, 0], gene_idx_array_X[:, 1])
            overlapping_genes.append(Agam_genes.loc[Agam_genes['Start'].isin(gene_idx_array_X[loc_ranges][:, 0])]['GeneID'])
overlapping_genes  

[8481    AGAP029667
 Name: GeneID, dtype: object,
 8930    AGAP004835
 Name: GeneID, dtype: object,
 8501    AGAP004827
 8714    AGAP004767
 Name: GeneID, dtype: object,
 8501    AGAP004827
 Name: GeneID, dtype: object,
 8804    AGAP004795
 Name: GeneID, dtype: object,
 Series([], Name: GeneID, dtype: object),
 8930    AGAP004835
 9026    AGAP004823
 Name: GeneID, dtype: object,
 Series([], Name: GeneID, dtype: object),
 8282    AGAP013689
 8398    AGAP004824
 Name: GeneID, dtype: object,
 8398    AGAP004824
 8930    AGAP004835
 Name: GeneID, dtype: object,
 6624    AGAP004846
 Name: GeneID, dtype: object,
 6620    AGAP004829
 9148    AGAP004865
 Name: GeneID, dtype: object,
 Series([], Name: GeneID, dtype: object),
 Series([], Name: GeneID, dtype: object),
 6721    AGAP005076
 9283    AGAP005137
 Name: GeneID, dtype: object,
 9289    AGAP005155
 Name: GeneID, dtype: object,
 6752    AGAP005160
 9078    AGAP005156
 Name: GeneID, dtype: object,
 6756    AGAP005170
 Name: GeneID, dtype: 

In [910]:
#use join to put together the genes in each region to comma separaed
overlapping_genes_for_df = []
for thing in overlapping_genes:
    if thing.values.any():
        gene_string = ','.join(thing.values)
        overlapping_genes_for_df.append(str(gene_string))
    else:
        overlapping_genes_for_df.append('NA')
overlapping_genes_for_df

['AGAP029667',
 'AGAP004835',
 'AGAP004827,AGAP004767',
 'AGAP004827',
 'AGAP004795',
 'NA',
 'AGAP004835,AGAP004823',
 'NA',
 'AGAP013689,AGAP004824',
 'AGAP004824,AGAP004835',
 'AGAP004846',
 'AGAP004829,AGAP004865',
 'NA',
 'NA',
 'AGAP005076,AGAP005137',
 'AGAP005155',
 'AGAP005160,AGAP005156',
 'AGAP005170',
 'AGAP005213',
 'AGAP005245',
 'AGAP005288,AGAP005281',
 'AGAP005285',
 'AGAP005341,AGAP005359',
 'AGAP005374',
 'AGAP005360',
 'NA',
 'AGAP005556,AGAP005536',
 'AGAP005623,AGAP005617',
 'AGAP005675',
 'NA',
 'NA',
 'NA',
 'AGAP005839',
 'AGAP005956',
 'AGAP005953,AGAP005961',
 'AGAP005953,AGAP005961',
 'AGAP006111',
 'NA',
 'AGAP006115,AGAP029121',
 'AGAP006116,AGAP029121',
 'AGAP029121',
 'AGAP029121',
 'AGAP006132,AGAP006131,AGAP006115',
 'AGAP006133,AGAP006115',
 'AGAP006132,AGAP006131',
 'AGAP006133',
 'AGAP029236,AGAP029656',
 'AGAP029656',
 'NA',
 'AGAP006328',
 'AGAP006328',
 'AGAP006330',
 'AGAP006330,AGAP006339',
 'AGAP006330',
 'AGAP006330,AGAP006345',
 'NA',
 'AGAP

In [911]:
#length of sweep signals in old sweeps analysis
mean_length_gambiae_sweeps = round(np.mean(all_gambiae_signals_df['new_focus_pstop']-all_gambiae_signals_df['new_focus_pstart']))
#mean_length_old_sweeps
mean_length_gambiae_sweeps

256866

In [912]:
#the new focussed sweep coordinates are actually wider than the old ones!
mean_length_old_sweeps

194810

In [913]:
#now need to append this to a dataframe of regions
#make a new dataframe and add this in - it seems to prefer being concatenated on the 1st axis
gambiae_sweeps_overlapping_genes_df = all_gambiae_signals_df[['cohort_id','new_contig','new_focus_pstart','new_focus_pstop']]
#regions_and_overlapping_genes_df['overlapping_genes'] = overlapping_genes_for_df
gambiae_sweeps_overlapping_genes_df = pd.concat((gambiae_sweeps_overlapping_genes_df, pd.DataFrame(overlapping_genes_for_df)), axis =1, ignore_index=True)
gambiae_sweeps_overlapping_genes_df.columns =['cohort_id','new_contig','new_focus_pstart','new_focus_pstop','overlapping_genes']
gambiae_sweeps_overlapping_genes_df

,cohort_id,new_contig,new_focus_pstart,new_focus_pstop,overlapping_genes
0,CF-BGF_Bangui_gamb_1994_Q1,2L,2110027,3324653,AGAP029667
1,CM-ES_Lom-Et-Djérem_gamb_2009_Q3,2L,2862237,4287745,AGAP004835
2,CM-ES_Haut-Nyong_gamb_2009_Q3,2L,3214444,4142772,"AGAP004827,AGAP004767"
3,BF-09_Houet_gamb_2012_Q3,2L,3573248,4135887,AGAP004827
4,GN-N_Macenta_gamb_2012_Q4,2L,3619310,4103431,AGAP004795
...,...,...,...,...,...
217,CM-ES_Lom-Et-Djérem_gamb_2009_Q3,X,15421109,15472386,"AGAP000834,AGAP013506"
218,CF-BGF_Bangui_gamb_1994_Q1,X,15427476,15475943,AGAP013506
219,BF-09_Houet_gamb_2012_Q3,X,20499760,23324523,NA
220,ML-3_Yanfolila_gamb_2012_Q4,X,20552409,23054237,NA


In [914]:
import re

In [915]:
def find_match_count(word: str, pattern: str) -> int:
    return len(re.findall(pattern, word.upper()))

In [916]:
#this does not seem to work however for the overlapping genes! Why? need to use .upper in fuction as AGAP is upper
string_list = ['AGAP']
for col in string_list:
    print(col)
    gambiae_sweeps_overlapping_genes_df[col] = gambiae_sweeps_overlapping_genes_df['overlapping_genes'].apply(find_match_count, pattern=col)
gambiae_sweeps_overlapping_genes_df

AGAP


,cohort_id,new_contig,new_focus_pstart,new_focus_pstop,overlapping_genes,AGAP
0,CF-BGF_Bangui_gamb_1994_Q1,2L,2110027,3324653,AGAP029667,1
1,CM-ES_Lom-Et-Djérem_gamb_2009_Q3,2L,2862237,4287745,AGAP004835,1
2,CM-ES_Haut-Nyong_gamb_2009_Q3,2L,3214444,4142772,"AGAP004827,AGAP004767",2
3,BF-09_Houet_gamb_2012_Q3,2L,3573248,4135887,AGAP004827,1
4,GN-N_Macenta_gamb_2012_Q4,2L,3619310,4103431,AGAP004795,1
...,...,...,...,...,...,...
217,CM-ES_Lom-Et-Djérem_gamb_2009_Q3,X,15421109,15472386,"AGAP000834,AGAP013506",2
218,CF-BGF_Bangui_gamb_1994_Q1,X,15427476,15475943,AGAP013506,1
219,BF-09_Houet_gamb_2012_Q3,X,20499760,23324523,NA,0
220,ML-3_Yanfolila_gamb_2012_Q4,X,20552409,23054237,NA,0


In [917]:
#get some stats on numbers of overlapping genes
max(gambiae_sweeps_overlapping_genes_df['AGAP'])

3

In [918]:
np.mean(gambiae_sweeps_overlapping_genes_df['AGAP'])

1.0585585585585586

In [919]:
#check the region for vgsc more carefully AGAP004707 which has positions 2358158 to 2431617 on 2L
#first sort the dataframe of signals by ascending values to see wwhat is in that region
gambiae_sweeps_overlapping_genes_sorted = gambiae_sweeps_overlapping_genes_df.sort_values(['new_contig', 'new_focus_pstart'], ascending = [True, True])

In [849]:
#it looks like vgsc (AGAP004707) should have come up in the first row below - but it doesn't. Something has gone wrong
gambiae_sweeps_overlapping_genes_sorted[gambiae_sweeps_overlapping_genes_sorted['new_contig']=='2L']

,cohort_id,new_contig,new_focus_pstart,new_focus_pstop,overlapping_genes,AGAP
120,CF-BGF_Bangui_gamb_1994_Q1,2L,2110027,3324653,"AGAP002858,AGAP002867,AGAP002859",3
71,CM-ES_Lom-Et-Djérem_gamb_2009_Q3,2L,2862237,4287745,AGAP006330,1
50,CM-ES_Haut-Nyong_gamb_2009_Q3,2L,3214444,4142772,AGAP005285,1
186,BF-09_Houet_gamb_2012_Q3,2L,3573248,4135887,AGAP009189,1
134,GN-N_Macenta_gamb_2012_Q4,2L,3619310,4103431,AGAP029534,1
...,...,...,...,...,...,...
190,BF-09_Houet_gamb_2012_Q3,2L,43863637,43907464,AGAP009193,1
215,ML-3_Yanfolila_gamb_2012_Q4,2L,43900324,43932302,NA,0
23,UG-E_Tororo_gamb_2012_Q4,2L,44164910,44220273,AGAP005170,1
141,GN-N_Macenta_gamb_2012_Q4,2L,47392285,47417314,AGAP029534,1
